In [3]:
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import math
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
HOME = "https://www.youtube.com/user/theneedledrop/videos"

In [4]:
def scroll():
    """
    scrolls to the bottom of the page
    code taken from https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python
    modified slightly for YouTube
    """
    SCROLL_PAUSE_TIME = 0.5

    last_height = driver.execute_script("return window.scrollY")

    tries = 0
    while True:
        down_height = last_height + 1000
        driver.execute_script("window.scrollTo(0," + str(down_height)  + ")")

        time.sleep(SCROLL_PAUSE_TIME)

        new_height = driver.execute_script("return window.scrollY")
        if new_height == last_height:
            tries += 1
            time.sleep(SCROLL_PAUSE_TIME)
            if tries == 50:
                break
        else:
            tries = 0
        last_height = new_height

In [25]:
#Scrape for videos
# WARNING: Takes very long
driver = webdriver.Chrome("C:\webdriver\chromedriver.exe")
driver.get(HOME)

scroll()
element_titles = driver.find_elements(By.ID,"video-title")

AttributeError: 'DataFrame' object has no attribute 'Chrome'

In [20]:
len(element_titles)

3955

In [24]:
element_titles[0].get_property('attributes')[0]

{'ATTRIBUTE_NODE': 2,
 'CDATA_SECTION_NODE': 4,
 'COMMENT_NODE': 8,
 'DOCUMENT_FRAGMENT_NODE': 11,
 'DOCUMENT_NODE': 9,
 'DOCUMENT_POSITION_CONTAINED_BY': 16,
 'DOCUMENT_POSITION_CONTAINS': 8,
 'DOCUMENT_POSITION_DISCONNECTED': 1,
 'DOCUMENT_POSITION_FOLLOWING': 4,
 'DOCUMENT_POSITION_IMPLEMENTATION_SPECIFIC': 32,
 'DOCUMENT_POSITION_PRECEDING': 2,
 'DOCUMENT_TYPE_NODE': 10,
 'ELEMENT_NODE': 1,
 'ENTITY_NODE': 6,
 'ENTITY_REFERENCE_NODE': 5,
 'NOTATION_NODE': 12,
 'PROCESSING_INSTRUCTION_NODE': 7,
 'TEXT_NODE': 3,
 '__shady_addEventListener': {},
 '__shady_appendChild': {},
 '__shady_childNodes': [],
 '__shady_cloneNode': {},
 '__shady_contains': {},
 '__shady_dispatchEvent': {},
 '__shady_firstChild': None,
 '__shady_getRootNode': {},
 '__shady_insertBefore': {},
 '__shady_isConnected': False,
 '__shady_lastChild': None,
 '__shady_native_addEventListener': {},
 '__shady_native_appendChild': {},
 '__shady_native_childNodes': [],
 '__shady_native_cloneNode': {},
 '__shady_native_contain

Look into Pytube for library

In [16]:
#Creation of the data
GENRE_PATTERN = re.compile("[^\/](19|20)\d{2}\s\/.*")
PATTERN = re.compile('(?<=shortDescription":").*(?=","isCrawlable)')
musicdataDF = pd.DataFrame(columns=["Description"])
for element in element_titles:
    title = element.text
    print(title)
    print( element.get_attribute("href"))
    print( requests.get(element.get_attribute("href")))
#Not Good and Classics are included within this data
    try:
        if "review" in title.lower() and "track review" not in title.lower() and "yunoreview" not in title.lower() and "weekly track" not in title.lower():
            #Get Description of video
            soup = BeautifulSoup(requests.get(element.get_attribute("href")).content)
            description = PATTERN.findall(str(soup))[0].replace('\\n','\n').lower()
            print(description)
            row = {"Description":description}
            musicdataDF = musicdataDF.append(row,ignore_index=True)
    except Exception:
        pass

Denzel Curry, MGK, JID, AJR | Weekly Track Roundup: 11/28/22
None


MissingSchema: Invalid URL 'None': No scheme supplied. Perhaps you meant http://None?

In [ ]:
title = element.text
    print(title)
    print( element.get_attribute("href"))
    print( requests.get(element.get_attribute("href")))
#Not Good and Classics are included within this data
    try:
        if "review" in title.lower() and "track review" not in title.lower() and "yunoreview" not in title.lower() and "weekly track" not in title.lower():
            #Get Description of video
            soup = BeautifulSoup(requests.get(element.get_attribute("href")).content)
            description = PATTERN.findall(str(soup))[0].replace('\\n','\n').lower()
            print(description)
            row = {"Description":description}
            musicdataDF = musicdataDF.append(row,ignore_index=True)
    except Exception:
        pass

In [ ]:
display(musicdataDF)

In [ ]:
            #Get Score of the video
            score = re.search("[^201]\n\d{0,2}\/10", description).group().strip("\n").split("/")[0] #Prevents word classification to not be included (Classic, meh, Not Good)

            #Get the Genre
            genre = GENRE_PATTERN.search(description).group()

            #Get the year the album dropped
            album_date = GENRE_PATTERN.search(description).group().split("/")[0].strip()

            #Date of when the video was published
            video_date = soup.find("meta", itemprop="datePublished")['content']

In [ ]:
#Save the music data to csv
music_data.to_csv("..\NeedleDrop\music_data.csv")

# Data Cleaning

In [ ]:
#Fixed the data frame
music_data = pd.read_csv(".\music_data.csv", index_col=0,encoding='latin-1')

In [ ]:

#Fix Score Counts and convert to numerics
music_data["Score"] = music_data["Score"].str.replace(".\\n","")
music_data["Score"] = pd.to_numeric(music_data["Score"])

In [ ]:
#Remove "{Name} Review From Title"
pattern_album = re.compile("(ALBUM|EP|MIXTAPE) .*", re.I)
music_data["Title"] = music_data["Title"].str.replace(pattern_album,"")
music_data["Title"] = music_data["Title"].str.replace("EP/","") #SZA- Z man

In [ ]:
#Splitting title into artist and album
music_data[["Artist", "Album"]] = music_data["Title"].str.split(pat = "-",expand=True, n = 1)
del music_data["Title"]
music_data = music_data[["Artist", "Album", "Genre", "Album_Date", "Video_Date", "Score"]]

In [ ]:
#Filter the sub-genre to the parent-genre
music_data["Genre"].mask( music_data["Genre"].str.contains("r&b|soul|funk",case=False, na=False), "R&B", inplace=True )
music_data["Genre"].mask( music_data["Genre"].str.contains("hip hop|rap|trap|drill",case=False, na=False), "Rap", inplace=True )
music_data["Genre"].mask( music_data["Genre"].str.contains("rock|punk|emo|screamo|grunge|metal|hardcore",case=False, na=False), "Rock", inplace=True )
music_data["Genre"].mask( music_data["Genre"].str.contains("jazz|bop",case=False, na=False), "Jazz", inplace=True )
music_data["Genre"].mask( music_data["Genre"].str.contains("electro|dub|synth|house|dance|disco|edm|glitch|idm",case=False, na=False), "Electronic", inplace=True )
music_data["Genre"].mask( music_data["Genre"].str.contains("ambient",case=False, na=False), "Ambient", inplace=True )
music_data["Genre"].mask( music_data["Genre"].str.contains("pop",case=False, na=False), "Pop", inplace=True )
music_data["Genre"].mask( music_data["Genre"].str.contains("folk",case=False, na=False), "Folk", inplace=True )
music_data["Genre"].mask( music_data["Genre"].str.contains("country",case=False, na=False), "Country", inplace=True )

In [ ]:
import dataframe_image as dfi
artists = music_data["Artist"].str.replace(" ","")
artists.value_counts().head(n=5)

In [ ]:
print("Mr.Fantano has reviewed about " + str(music_data.shape[0]) + " albums/ep/mixtape as of 5/22/2022 => 1/21/2011 ") #Not including classics, Not Good, and

In [ ]:
music_data["Video_Date"] = pd.to_datetime(music_data["Video_Date"])
music_data["Video_Date"].dt.year.value_counts().sort_index().plot(kind="bar",xlabel="Years of Upload", ylabel="Counts",title="Reviews Throughout Years", figsize=(10,5))
print( "Mr.Fantano has been uploading reviews since 2011 and on average releases " +
       str( math.ceil( music_data["Video_Date"].dt.year.value_counts().mean() ) ) +" reviews per year")

## Bar Chart w/ all scores

In [ ]:
music_data["Score"].value_counts().sort_index().plot(kind="bar",xlabel="Scores", ylabel="Counts",title="All Scores", figsize=(10,5))
music_data["Score"].describe()

# What is the average per genre? (Size > 30)

In [ ]:
music_data["Genre"].value_counts().head(n=6)

In [ ]:
#Rap
rap = music_data [music_data["Genre"] == "Rap"]
rap.reset_index(drop=True)
rap["Score"].value_counts().sort_index().plot(kind="bar",xlabel="Scores", ylabel="Counts",title="Rap Scores", figsize=(10,5))
print("Rap Scores:\n")
rap["Score"].describe()

In [ ]:
#Rock
rock = music_data [music_data["Genre"] == "Rock"]
rock.reset_index(drop=True)
rock["Score"].value_counts().sort_index().plot(kind="bar",xlabel="Scores", ylabel="Counts",title="Rock Scores", figsize=(10,5))
print("Rock Scores:\n")
rock["Score"].describe()

In [ ]:
#Pop
pop = music_data [music_data["Genre"] == "Pop"]
pop.reset_index(drop=True)
pop["Score"].value_counts().sort_index().plot(kind="bar",xlabel="Scores", ylabel="Counts",title="Pop Scores", figsize=(10,5))
print("Pop Scores:\n")
pop["Score"].describe()

In [ ]:
#R&B
RB = music_data [music_data["Genre"] == "R&B"]
RB.reset_index(drop=True)
RB["Score"].value_counts().sort_index().plot(kind="bar",xlabel="Scores", ylabel="Counts",title="R&B Scores", figsize=(10,5))
print("R&B Scores:\n")
RB["Score"].describe()

In [ ]:
electronic = music_data [music_data["Genre"] == "Electronic"]
electronic.reset_index(drop=True)
electronic["Score"].value_counts().sort_index().plot(kind="bar",xlabel="Scores", ylabel="Counts",title="Electronic Scores", figsize=(10,5))
print("Electronic Scores:\n")
electronic["Score"].describe()

In [ ]:
folk = music_data [music_data["Genre"] == "Folk"]
folk.reset_index(drop=True)
folk["Score"].value_counts().sort_index().plot(kind="bar",xlabel="Scores", ylabel="Counts",title="Folk Scores", figsize=(10,5))
print("Electronic Scores:\n")
folk["Score"].describe()

# Is there a bias within Mr.Fantano's review?
> There has been a large discussion online that Mr.Fantano doesn't like rap and is a big metal/rock fan

In [ ]:
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
f_stats, p_value = stats.f_oneway(rap["Score"],rock["Score"],pop["Score"],RB["Score"], electronic["Score"],folk["Score"])
if p_value < 0.05:
    print("Means are not equal " + str( format(p_value, '.10f')) + " Meaning that some bias is going on!\n")

combined = pd.concat([rap,rock,pop,RB,electronic,folk], axis = 0, ignore_index=True)

In [ ]:
poc_hoc = pairwise_tukeyhsd(endog = combined["Score"], groups = combined["Genre"] ,alpha=.05)
print(poc_hoc)